**Author:** Indronil Bhattacharjee

**Submitted on:** September 26, 2023


=============================================================


In [ ]:
# Load required libraries
library(dplyr)
library(ggplot2)

# Load the GFF3 file (replace 'path_to_gff3_file' with the actual file path)
gff3_file <- read.table("/kaggle/input/gencode-human-genome-annotation/gencode.v44.primary_assembly.annotation.gff3", header = FALSE, comment.char = "#", sep = "\t")

### **Task 1:** Linear interval search ###


In [ ]:
# Linear search to count overlapping features
count.features.linear <- function(chr, x, y, GFF) {
  count <- 0
  for (i in 1:nrow(GFF)) {
    if (GFF$chromosome[i] == chr && GFF$end[i] >= x && GFF$start[i] <= y) {
      count <- count + 1
    }
  }
  return(count)
}

### **Task 2:** Vectorized interval search ###


In [ ]:
# Vectorized search to count overlapping features
count.features.vectorized <- function(chr, x, y, GFF) {
  count <- sum(GFF$chromosome == chr & GFF$end >= x & GFF$start <= y)
  return(count)
}

### **Task 3:** Binary interval search ###


In [ ]:
count.features.binary <- function(chr, x, y, sorted.coordinates) {
  count <- 0
  left <- 1
  right <- length(sorted.coordinates)
  
  while (left <= right) {
    mid <- left + floor((right - left) / 2)
    if (sorted.coordinates[mid] >= x && sorted.coordinates[mid] <= y) {
      count <- count + 1
    }
    if (sorted.coordinates[mid] < x) {
      left <- mid + 1
    } else {
      right <- mid - 1
    }
  }
  return(count)
}

### **Task 4:** Reporting the runtime ###


In [ ]:
# Example usage and runtime reporting
chr <- "chr1"
x <- 10000
y <- 12000
# Extract relevant data for the chromosome
# Assuming the GFF3 format columns: chromosome, start, end
GFF <- gff3_file %>%
  filter(V1 == chr) %>%
  select(chromosome = V1, start = V4, end = V5) %>%
  mutate(start = as.numeric(start), end = as.numeric(end))  # Convert start and end to numeric

# Sort the coordinates for binary search
sorted.coordinates <- sort(c(GFF$start, GFF$end))

# Runtime for linear search
linear_time <- system.time(count.features.linear(chr, x, y, GFF))[3]
linear_result <- count.features.linear(chr, x, y, GFF)
cat("Linear Search:", linear_result, "times\n")

# Runtime for vectorized search
vectorized_time <- system.time(count.features.vectorized(chr, x, y, GFF))[3]
vectorized_result <- count.features.vectorized(chr, x, y, GFF)
cat("Vectorized Search:", vectorized_result, "times\n")

# Runtime for binary search
binary_time <- system.time(count.features.binary(chr, x, y, sorted.coordinates))[3]
binary_result <- count.features.binary(chr, x, y, sorted.coordinates)
cat("Binary Search:", binary_result, "times\n")

# Report runtimes
cat("Linear Search Runtime:", linear_time, "seconds\n")
cat("Vectorized Search Runtime:", vectorized_time, "seconds\n")
cat("Binary Search Runtime:", binary_time, "seconds\n")

**4.1 Visualization of runtimes**


In [ ]:
# Create a vector of runtimes
runtimes <- c(linear_time, vectorized_time, binary_time)

# Create a bar chart
barplot(runtimes, 
        names.arg = c("Linear Search", "Vectorized Search", "Binary Search"),
        col = c("blue", "red", "green"),  # Colors for the bars
        ylab = "Runtime (seconds)",  # Label for the y-axis
        main = "Comparison of Runtimes",  # Title of the plot
        ylim = c(0, max(runtimes) + 0.01))  # Adjust the y-axis limits

# Extra Tasks


### **Task E1:** Query interval spanning only known coordinates ###


In [ ]:
annotation_intervals <- list()
for (i in 1:nrow(GFF)){
    pair <- c(GFF$start[i], GFF$end[i])
    annotation_intervals <- c(annotation_intervals, list(pair))
}

In [ ]:
# Initialize a list of pairs to store intervals like dictionary
interval_map <- list()

# Populate the hashmap with interval coordinates
for (idx in seq_along(annotation_intervals)) {
  interval <- annotation_intervals[[idx]]
  start <- interval[1]
  end <- interval[2]
  for (coord in seq(start, end)) {
    interval_map[[as.character(coord)]] <- idx
  }
}

In [ ]:
# Function to find overlapping intervals for [x, y]
find_overlapping_intervals <- function(x, y, interval_map) {
  count <- 0
  for (coord in seq(x, y)) {
    if (as.character(coord) %in% names(interval_map)) {
      count <- count + 1
    }
  }
  return(count)
}

# Query interval
x <- 5788
y <- 5824

# Find overlapping intervals

count <- find_overlapping_intervals(x, y, interval_map)
runtime_imp <- system.time(find_overlapping_intervals(x, y, interval_map))[3]

# Print the result
cat(count, "times\n")
cat("Improved runtime: ", runtime_imp, "seconds")